# NYC Clustering Problem

# use case explanation

We are familiar with New York City taxi dataset as we did some analytics over it to answer some questions. Now it is your turn to answer the following question Which manhattan (our interest borough) area should a Taxi driver choose to get a high tip?

for improving the usefulness of the service and address the use case better. We need to answer the question for an example
Which Manhattan area should a Taxi driver choose to get a high tip? we need to know a more precise location inside manhattan, not just neighborhood. 

An area could be any zone of arbitrary size. you need to search for small zones with elevated tip.  Each such an area is a cluster of Taxi rides with elevated tips.

In [3]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.rdd import portable_hash
from pyspark.statcounter import StatCounter
from pyspark.sql.types import StringType,IntegerType,FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import to_timestamp, current_timestamp, col,expr,unix_timestamp,round,when,hour

import os
import json

from datetime import datetime
from matplotlib import pyplot as plt

In [4]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Taxi")\
        .config("spark.driver.memory", "4g")\
        .config("spark.driver.cores", "4")\
        .getOrCreate()
sc=spark.sparkContext

# Use Schemas below for reading the two dataframes

In [6]:
from pyspark.sql.types import StructField,StructType,LongType,FloatType,ShortType,TimestampType,StringType

taxiRidesSchema = StructType([StructField("rideId", LongType()), StructField("isStart", StringType()),\
                             StructField("endTime", TimestampType()), StructField("startTime", TimestampType()),\
                             StructField("startLon", FloatType()), StructField("startLat", FloatType()),\
                             StructField("endLon", FloatType()), StructField("endLat", FloatType()),\
                             StructField("passengerCnt", ShortType()), StructField("taxiId", LongType()),\
                             StructField("driverId", LongType())])
                  


In [7]:
taxiFaresSchema = StructType([StructField("rideId", LongType()), StructField("taxiId", LongType()),\
                                 StructField("driverId", LongType()), StructField("startTime", TimestampType()),\
                                 StructField("paymentType", StringType()), StructField("tip", FloatType()),\
                                 StructField("tolls", FloatType()), StructField("totalFare", FloatType())])
      

# Reading the datasets

In [9]:
#Taxi Ride
dfR = spark.read.option("header", True).option("inferSchema",True).schema(taxiRidesSchema).csv("/FileStore/tables/nycTaxiRides.csv")
dfR.show(1)

+------+-------+-------------------+-------------------+----------+--------+---------+--------+------------+----------+----------+
rideId|isStart| endTime| startTime| startLon|startLat| endLon| endLat|passengerCnt| taxiId| driverId|
+------+-------+-------------------+-------------------+----------+--------+---------+--------+------------+----------+----------+
 11| START|2013-01-01 00:00:00|1970-01-01 00:00:00|-73.870834|40.77377|-73.79236|40.77176| 1|2013000011|2013000011|
+------+-------+-------------------+-------------------+----------+--------+---------+--------+------------+----------+----------+
only showing top 1 row

In [10]:
#Taxi fare
df = spark.read.option("header", True).option("inferSchema",True).schema(taxiFaresSchema).csv("/FileStore/tables/nycTaxiFares.csv")
df.show(1) 

+------+----------+----------+-------------------+-----------+---+-----+---------+
rideId| taxiId| driverId| startTime|paymentType|tip|tolls|totalFare|
+------+----------+----------+-------------------+-----------+---+-----+---------+
 2|2013000002|2013000002|2013-01-01 00:00:00| CSH|0.0| 0.0| 7.0|
+------+----------+----------+-------------------+-----------+---+-----+---------+
only showing top 1 row

# Preparation and the Model

In [12]:
from pyspark.sql.functions import *
dfR1=dfR.withColumn("year", year(col("startTime"))).withColumn("month", month(col("startTime"))).withColumn("day", dayofmonth(col("startTime")))

In [13]:
dff= df.withColumn("year",year(col("startTime"))).withColumn("month",month(col("startTime"))).withColumn("day", dayofmonth(col("startTime")))

In [14]:
dfRide =dfR1.select("rideId","endTime","startLon","startLat","endLon", "endLat")

In [15]:
dff =dff.select(col("rideId").alias("rideId_fares"), "startTime", "tip")

In [16]:
lonEast = -73.887
lonWest = -74.037
latNorth = 40.899
latSouth = 40.695

In [17]:
dfRide=dfRide.filter(col("startLon") >= lonWest)
dfRide=dfRide.filter(col("startLon") <= lonEast)
dfRide=dfRide.filter(col("startLat")  >= latSouth)
dfRide=dfRide.filter(col("startLat") <= latNorth)
dfRide=dfRide.filter(col("endLon") >= lonWest)
dfRide=dfRide.filter(col("endLon") <= lonEast)
dfRide=dfRide.filter(col("endLat") >= latSouth)
dfRide=dfRide.filter(col("endLat") <= latNorth)
dfRide=dfRide.filter(col("isStart") == "END")

In [18]:
dfRide.show(5)

+------+-------------------+----------+---------+----------+---------+
rideId| endTime| startLon| startLat| endLon| endLat|
+------+-------------------+----------+---------+----------+---------+
 65|2013-01-01 00:00:36| -73.99221|40.725124|-73.991646|40.726658|
 77|2013-01-01 00:01:22| -73.9701|40.768005| -73.96977|40.767834|
 70|2013-01-01 00:01:37| -73.97544|40.749657| -73.97733| 40.75199|
 94|2013-01-01 00:01:40|-74.005165| 40.72053| -74.00393|40.725655|
 53|2013-01-01 00:02:00| -73.95653|40.716976| -73.96244|40.715008|
+------+-------------------+----------+---------+----------+---------+
only showing top 5 rows

In [19]:
dfRide.count()

Out[16]: 136647

In [20]:
dfRidesManhattan =dfRide.join(dff, expr ("""rideId_fares = rideId AND endTime > startTime AND endTime <= startTime + interval 2 hours"""))

In [21]:
dfRidesManhattan=dfRidesManhattan.filter(col("tip") > 0)

In [22]:
dfRidesManhattan=dfRidesManhattan.withColumn("startHour", hour(col("startTime"))).drop(col("rideId_fares"))

In [23]:
dfRidesManhattan.show(5)

+------+-------------------+----------+---------+---------+---------+-------------------+---+---------+
rideId| endTime| startLon| startLat| endLon| endLat| startTime|tip|startHour|
+------+-------------------+----------+---------+---------+---------+-------------------+---+---------+
 120|2013-01-01 00:02:00| -74.0032|40.708313|-74.00561| 40.70655|2013-01-01 00:01:00|0.8| 0|
 9|2013-01-01 00:02:44| -73.99988|40.743343|-74.00371| 40.74828|2013-01-01 00:00:00|1.0| 0|
 47|2013-01-01 00:03:00| -73.99061|40.734997|-73.98949|40.730324|2013-01-01 00:00:00|0.9| 0|
 286|2013-01-01 00:03:00|-73.981544|40.781475|-73.97944|40.784386|2013-01-01 00:02:00|0.7| 0|
 109|2013-01-01 00:03:21| -73.96265|40.767467|-73.95406|40.776943|2013-01-01 00:00:55|1.5| 0|
+------+-------------------+----------+---------+---------+---------+-------------------+---+---------+
only showing top 5 rows

In [24]:
from pyspark.sql.types import DoubleType
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql import functions as F
from pyspark.sql.functions import udf

In [25]:
vectCol=udf(lambda tip : Vectors.dense(tip), VectorUDT())

In [26]:
 dfRidesManhattan=dfRidesManhattan.withColumn("tipVect", vectCol(dfRidesManhattan["tip"]))

In [27]:
dfRidesManhattan.show(3)

+------+-------------------+---------+---------+---------+---------+-------------------+---+---------+--------------------+
rideId| endTime| startLon| startLat| endLon| endLat| startTime|tip|startHour| tipVect|
+------+-------------------+---------+---------+---------+---------+-------------------+---+---------+--------------------+
 120|2013-01-01 00:02:00| -74.0032|40.708313|-74.00561| 40.70655|2013-01-01 00:01:00|0.8| 0| [0.800000011920929]|
 9|2013-01-01 00:02:44|-73.99988|40.743343|-74.00371| 40.74828|2013-01-01 00:00:00|1.0| 0| [1.0]|
 47|2013-01-01 00:03:00|-73.99061|40.734997|-73.98949|40.730324|2013-01-01 00:00:00|0.9| 0|[0.8999999761581421]|
+------+-------------------+---------+---------+---------+---------+-------------------+---+---------+--------------------+
only showing top 3 rows

In [28]:
from pyspark.ml.feature import MinMaxScaler
tipScaler =MinMaxScaler().setInputCol("tipVect").setOutputCol("tipScaled").setMin(0).setMax(1)

In [29]:
from pyspark.sql.functions import *
import numpy as np
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler, VectorSlicer
from pyspark.ml import Pipeline

In [30]:
featuresAssembler=VectorAssembler().setInputCols(['startLon', 'startLat', 'tip']).setOutputCol("features")

In [31]:
from pyspark.ml.clustering import KMeans,KMeansModel
from pyspark.ml.evaluation import ClusteringEvaluator

Ks1=[]
for i in range(7,16):
  kmeansEstimator=KMeans().setK(i).setSeed(1)
  prepareKmeansPipeline=Pipeline().setStages([tipScaler, featuresAssembler, kmeansEstimator])
  model_1 = prepareKmeansPipeline.fit(dfRidesManhattan)
  predictions_1 = model_1.transform(dfRidesManhattan)
  evaluator=ClusteringEvaluator()
  evaluator_1 = evaluator.evaluate(predictions_1)
  Ks1.append([i,evaluator_1])
Ks1 

Out[30]: [[7, 0.7059244720138791],
 [8, 0.7160366594963219],
 [9, 0.6525232786123568],
 [10, 0.7233564487238731],
 [11, 0.7321203730727127],
 [12, 0.7460985718755453],
 [13, 0.7435738783293344],
 [14, 0.7367343010126564],
 [15, 0.7286194624011337]]

In [32]:
res = sorted(Ks1, key = lambda i: i[1], reverse = True)[0][0]
 
# printing result 
print ("The best k for knn is : " , res)

The best k for knn is : 12

In [33]:
from pyspark.ml.clustering import KMeans,KMeansModel
kmeansEstimator=KMeans().setK(12).setSeed(1)
prepareKmeansPipeline=Pipeline().setStages([tipScaler, featuresAssembler, kmeansEstimator])
model_1 = prepareKmeansPipeline.fit(dfRidesManhattan)

In [34]:
centers = model_1.stages[2].clusterCenters()
for center in centers:
  print(center)

[-73.98098373 40.75077432 0.99231867]
[-73.98153251 40.75201634 39.76857158]
[-73.98139132 40.75468148 7.23585985]
[-73.98232455 40.75021428 3.82237045]
[-73.97148132 40.75501871 17.67999995]
[-73.98165225 40.74835951 1.51357339]
[-73.98167756 40.75251531 4.74939114]
[-73.97958481 40.75518137 10.18421875]
[-73.98139921 40.75026107 3.0586143 ]
[-73.9817902 40.75001082 2.50414194]
[-73.98118929 40.7538809 5.73060285]
[-73.98128569 40.75052028 2.00749863]

In [35]:
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator
Ks2=[]
for i in range(7,16):
  BisectingKMeansEstimator=BisectingKMeans().setK(i).setSeed(1)
  prepareKmeansPipeline=Pipeline().setStages([tipScaler, featuresAssembler, BisectingKMeansEstimator])
  model_2 = prepareKmeansPipeline.fit(dfRidesManhattan)
  predictions_2 = model_2.transform(dfRidesManhattan)
  evaluator=ClusteringEvaluator()
  evaluator_2 = evaluator.evaluate(predictions_2)
  Ks2.append([i,evaluator_2])
Ks2

Out[51]: [[7, 0.6812938549860327],
 [8, 0.6879899355359275],
 [9, 0.632998789545469],
 [10, 0.5244952298524915],
 [11, 0.5998549294322795],
 [12, 0.5785867385784536],
 [13, 0.6155966851932462],
 [14, 0.6277834390019988],
 [15, 0.6317736515032167]]

In [36]:
res = sorted(Ks2, key = lambda i: i[1], reverse = True)[0][0]
 
# printing result 
print ("The best k for BisectingKMeans is : " , res)

The best k for BisectingKMeans is : 8

In [37]:
from pyspark.ml.clustering import BisectingKMeans
BisectingKMeansEstimator=BisectingKMeans().setK(8).setSeed(1)
prepareKmeansPipeline=Pipeline().setStages([tipScaler, featuresAssembler, BisectingKMeansEstimator])
model_2 = prepareKmeansPipeline.fit(dfRidesManhattan)

In [38]:
from pyspark.ml.clustering import GaussianMixture

Ks3=[]
for i in range(7,16):
  GaussianMixtureEstimator=GaussianMixture().setK(i).setSeed(1)
  prepareKmeansPipeline=Pipeline().setStages([tipScaler, featuresAssembler, GaussianMixtureEstimator])
  model_3 = prepareKmeansPipeline.fit(dfRidesManhattan)
  predictions_3 = model_3.transform(dfRidesManhattan)
  evaluator=ClusteringEvaluator()
  evaluator_3 = evaluator.evaluate(predictions_3)
  Ks3.append([i,evaluator_2])
Ks3

Out[60]: [[7, 0.6879899355359275],
 [8, 0.6879899355359275],
 [9, 0.6879899355359275],
 [10, 0.6879899355359275],
 [11, 0.6879899355359275],
 [12, 0.6879899355359275],
 [13, 0.6879899355359275],
 [14, 0.6879899355359275],
 [15, 0.6879899355359275]]

In [39]:
res = sorted(Ks3, key = lambda i: i[1], reverse = True)[0][0]
 
# printing result 
print ("The best k for GaussianMixture is : " , res)

The best k for GaussianMixture is : 7

In [40]:
from pyspark.ml.clustering import GaussianMixture
GaussianMixtureEstimator=GaussianMixture().setK(7).setSeed(1)
prepareKmeansPipeline=Pipeline().setStages([tipScaler, featuresAssembler, GaussianMixtureEstimator])
model_3 = prepareKmeansPipeline.fit(dfRidesManhattan)


# Evaluating clusters
use ClusteringEvaluator

In [42]:
from pyspark.ml.evaluation import ClusteringEvaluator
predictions_1 = model_1.transform(dfRidesManhattan)
evaluator=ClusteringEvaluator()
evaluator_1 = evaluator.evaluate(predictions_1)
evaluator_1

Out[40]: 0.7460985718755453

In [43]:
#Computing cluster's variances
from pyspark.ml.stat import Summarizer

In [44]:
variances=predictions_1.groupBy(col("prediction")).agg(Summarizer.variance(col("features")).alias("clustersVar"))

In [45]:
type(variances)

Out[43]: pyspark.sql.dataframe.DataFrame

In [46]:
from pyspark.sql.types import FloatType
from pyspark.sql.types import DoubleType
from pyspark.ml.linalg import Vectors, VectorUDT

firstelement=udf(lambda v:float(v[2]),FloatType())
t=variances.select("prediction",firstelement('clustersVar').alias("tipvar"))

In [47]:
t.show()

+----------+-----------+
prediction| tipvar|
+----------+-----------+
 1| 67.782745|
 6| 0.07189813|
 3| 0.05272835|
 5|0.015664304|
 9|0.018636802|
 4| 9.595771|
 8|0.028891103|
 7| 1.0656937|
 10| 0.11889962|
 11|0.013068714|
 2| 0.3929267|
 0| 0.03311027|
+----------+-----------+

In [48]:
m=t.agg(max("tipvar")).collect()[0][0]
Zone_Elavated_Tip=t.filter(col("tipvar")==m).select("prediction").collect()[0][0]
print("The zone with elevated tip is : ",Zone_Elavated_Tip)

The zone with elevated tip is : 1

In [49]:
predictions_2 = model_2.transform(dfRidesManhattan)
evaluator=ClusteringEvaluator()
evaluator_2 = evaluator.evaluate(predictions_2)
evaluator_2

Out[55]: 0.6879899355359275

In [50]:
variances_2=predictions_2.groupBy(col("prediction")).agg(Summarizer.variance(col("features")).alias("clustersVar"))
firstelement=udf(lambda v:float(v[2]),FloatType())
ff=variances_2.select("prediction",firstelement('clustersVar').alias("tipvar"))

In [51]:
ff.show()

+----------+-----------+
prediction| tipvar|
+----------+-----------+
 1|0.026348636|
 6| 8.28897|
 3|0.066539735|
 5| 0.90482646|
 4| 0.28500912|
 7| 67.782745|
 2| 0.03613|
 0| 0.03773419|
+----------+-----------+

In [52]:
mm=ff.agg(max("tipvar")).collect()[0][0]
Zone_Elavated_Tip2=ff.filter(col("tipvar")==mm).select("prediction").collect()[0][0]
print("The zone with elevated tip is : ",Zone_Elavated_Tip2)

The zone with elevated tip is : 7

In [53]:
from pyspark.ml.evaluation import ClusteringEvaluator
predictions_3 = model_3.transform(dfRidesManhattan)
evaluator=ClusteringEvaluator()
evaluator_3 = evaluator.evaluate(predictions_3)
evaluator_3

Out[65]: -0.04442835366342548

In [54]:
variances_3=predictions_3.groupBy(col("prediction")).agg(Summarizer.variance(col("features")).alias("clustersVar"))
firstelement=udf(lambda v:float(v[2]),FloatType())
kk=variances_3.select("prediction",firstelement('clustersVar').alias("tipvar"))

In [55]:
kk.show()

+----------+----------+
prediction| tipvar|
+----------+----------+
 1| 0.2409173|
 6| 2.0849001|
 3| 1.0037638|
 5| 168.71373|
 4| 4.368591|
 2| 1.5711657|
 0|0.67991567|
+----------+----------+

In [56]:
mmm=kk.agg(max("tipvar")).collect()[0][0]
Zone_Elavated_Tip3=kk.filter(col("tipvar")==mmm).select("prediction").collect()[0][0]
print("The zone with elevated tip is : ",Zone_Elavated_Tip3)

The zone with elevated tip is : 5